
## Cross-validate particular dataset

In [ ]:
import os
import sys
from collections import defaultdict

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    
from IPython.display import clear_output
from source.analysis import metrics
from source.dataloader import DataLoader
from source.trainer import Trainer
from source.trainer.models import import_model

In [ ]:
estimator_name = "savgol-svc"
do_optimize = True
group_id = 16
imaging_id  = [1, 2, 3]
camera_label = ["vnir", "swir"]

scores_all = defaultdict(list)
for seed in range(0,2):
    X, y, meta = DataLoader().load_datasets(
        group_id=group_id,
        imagings_ids=imaging_id,
        cameras_labels=camera_label,
        balancer_seed=seed,
    )

    trainer = Trainer(estimator_name)
    study = trainer.optimize(X, y)
    clear_output(wait=True)
        
    model = import_model(estimator_name)
    model.set_params(**study.best_params)

    scores = metrics.calculate_metrics(model, trainer.encoder, X, y, meta)
    clear_output(wait=True)

    filtered_scores = [metric for metric in scores if metric.meta_id == metrics.ALL]

    for metric in filtered_scores:
        scores_all[metric.name].append(metric.mean)
    